In [4]:
! pip install -q kaggle

In [5]:

import pandas as pd
from IPython.display import YouTubeVideo

In [7]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ericmpofu","key":"01e29d88e1577a9f6e2cc4da5f1d40ce"}'}

In [8]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [10]:
!kaggle datasets list

ref                                                                  title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
shreyanshverma27/online-sales-dataset-popular-marketplace-data       Online Sales Dataset - Popular Marketplace Data    7KB  2024-05-25 23:55:26           9455        167  1.0              
teocalvo/teomewhy-loyalty-system                                     TeoMeWhy Loyalty System                           23MB  2024-06-20 11:05:17            276        405  1.0              
zeesolver/data-eng-salary-2024                                       Data Developer Salary in 2024💰                   108KB  2024-06-10 07:33:22           1546         25  1.0              
sahilprajapati143/retail-analysis-large-dataset   

In [11]:

!kaggle datasets download -d skihikingkevin/pubg-match-deaths

Dataset URL: https://www.kaggle.com/datasets/skihikingkevin/pubg-match-deaths
License(s): CC0-1.0
100% 4.08G/4.10G [01:29<00:00, 90.4MB/s]
100% 4.10G/4.10G [01:29<00:00, 48.9MB/s]


In [12]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string)
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


In [13]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: test
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
    - city
    - price
    - distance

Writing file.yaml


In [14]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [15]:
config_data['inbound_delimiter']

','

In [16]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'test',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['city', 'price', 'distance']}

In [26]:
import pandas as pd
import zipfile
import time

# Define the path to the zipped file and the file within the zip
zip_file_path = '/content/pubg-match-deaths.zip'
file_name_within_zip = 'aggregate/agg_match_stats_0.csv'

# Start the timer
start_time = time.time()

# Open the zipped file
with zipfile.ZipFile(zip_file_path, 'r') as z:
    # Open the specific file within the zip
    with z.open(file_name_within_zip) as f:
        # Read the file into a Pandas DataFrame
        df = pd.read_csv(f)

# End the timer
end_time = time.time()

# Display the DataFrame
print(df)

# Print the elapsed time
print(f"Elapsed time: {end_time - start_time} seconds")



                              date  game_size  \
0         2017-11-26T20:59:40+0000         37   
1         2017-11-26T20:59:40+0000         37   
2         2017-11-26T20:59:40+0000         37   
3         2017-11-26T20:59:40+0000         37   
4         2017-11-26T20:59:40+0000         37   
...                            ...        ...   
13849282  2018-01-04T04:12:58+0000         46   
13849283  2018-01-04T04:12:58+0000         46   
13849284  2018-01-04T04:12:58+0000         46   
13849285  2018-01-04T04:12:58+0000         46   
13849286  2018-01-04T04:12:58+0000         46   

                                                   match_id match_mode  \
0         2U4GBNA0YmnNZYkzjkfgN4ev-hXSrak_BSey_YEG6kIuDG...        tpp   
1         2U4GBNA0YmnNZYkzjkfgN4ev-hXSrak_BSey_YEG6kIuDG...        tpp   
2         2U4GBNA0YmnNZYkzjkfgN4ev-hXSrak_BSey_YEG6kIuDG...        tpp   
3         2U4GBNA0YmnNZYkzjkfgN4ev-hXSrak_BSey_YEG6kIuDG...        tpp   
4         2U4GBNA0YmnNZYkzjkfgN4ev-hXSrak

In [20]:
!pip install dask[dataframe]

In [21]:
import os
import dask.dataframe as dd


import warnings
warnings.filterwarnings(action="ignore")

In [27]:
import dask.dataframe as dd
import zipfile
import time
import os
import tempfile

# Define the path to the zipped file and the file within the zip
zip_file_path = '/content/pubg-match-deaths.zip'
file_name_within_zip = 'aggregate/agg_match_stats_0.csv'

# Start the timer
start_time = time.time()

# Create a temporary directory
with tempfile.TemporaryDirectory() as temp_dir:
    # Extract the specific file within the zip to the temporary directory
    with zipfile.ZipFile(zip_file_path, 'r') as z:
        z.extract(file_name_within_zip, path=temp_dir)

    # Construct the path to the extracted file
    extracted_file_path = os.path.join(temp_dir, file_name_within_zip)

    # Read the extracted file into a Dask DataFrame, specifying dtypes
    df = dd.read_csv(extracted_file_path, dtype={'Cabin': 'object'})

    # Compute and display the DataFrame (this triggers actual computation)
    df_head = df.head()
    print(df_head)

    # End the timer
    end_time = time.time()

    # Print the elapsed time
    print(f"Elapsed time: {end_time - start_time} seconds")
    print(f'Number of Rows: {len(df)}, Number of Columns: {len(df.columns)}')


                       date  game_size  \
0  2017-11-26T20:59:40+0000         37   
1  2017-11-26T20:59:40+0000         37   
2  2017-11-26T20:59:40+0000         37   
3  2017-11-26T20:59:40+0000         37   
4  2017-11-26T20:59:40+0000         37   

                                            match_id match_mode  party_size  \
0  2U4GBNA0YmnNZYkzjkfgN4ev-hXSrak_BSey_YEG6kIuDG...        tpp           2   
1  2U4GBNA0YmnNZYkzjkfgN4ev-hXSrak_BSey_YEG6kIuDG...        tpp           2   
2  2U4GBNA0YmnNZYkzjkfgN4ev-hXSrak_BSey_YEG6kIuDG...        tpp           2   
3  2U4GBNA0YmnNZYkzjkfgN4ev-hXSrak_BSey_YEG6kIuDG...        tpp           2   
4  2U4GBNA0YmnNZYkzjkfgN4ev-hXSrak_BSey_YEG6kIuDG...        tpp           2   

   player_assists  player_dbno  player_dist_ride  player_dist_walk  \
0               0            1        2870.72400       1784.847780   
1               0            1        2938.40723       1756.079710   
2               0            0           0.00000        224.15

In [19]:
pip install ray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 MB 8.5 MB/s eta 0:00:00


In [24]:
import ray
import time
import zipfile
import os

ray.init()

# Define the path to the zipped file and the file within the zip
zip_file_path = '/content/pubg-match-deaths.zip'
file_name_within_zip = 'aggregate/agg_match_stats_0.csv'
# Start the timer
start_time = time.time()

# Define a remote function to read the CSV file
@ray.remote
def read_csv_remote(file_path):
    import pandas as pd
    with zipfile.ZipFile(file_path, 'r') as z:
        with z.open(file_name_within_zip) as f:
            df = pd.read_csv(f)
    return df

# Execute the remote function asynchronously
df_remote = read_csv_remote.remote(zip_file_path)

# Wait for the result
df = ray.get(df_remote)

# End the timer
end_time = time.time()

# Display the DataFrame
print(df)

# Print the elapsed time
print(f"Elapsed time: {end_time - start_time} seconds")
ray.shutdown()

2024-06-20 19:28:28,729	INFO worker.py:1753 -- Started a local Ray instance.


                              date  game_size  \
0         2017-11-26T20:59:40+0000         37   
1         2017-11-26T20:59:40+0000         37   
2         2017-11-26T20:59:40+0000         37   
3         2017-11-26T20:59:40+0000         37   
4         2017-11-26T20:59:40+0000         37   
...                            ...        ...   
13849282  2018-01-04T04:12:58+0000         46   
13849283  2018-01-04T04:12:58+0000         46   
13849284  2018-01-04T04:12:58+0000         46   
13849285  2018-01-04T04:12:58+0000         46   
13849286  2018-01-04T04:12:58+0000         46   

                                                   match_id match_mode  \
0         2U4GBNA0YmnNZYkzjkfgN4ev-hXSrak_BSey_YEG6kIuDG...        tpp   
1         2U4GBNA0YmnNZYkzjkfgN4ev-hXSrak_BSey_YEG6kIuDG...        tpp   
2         2U4GBNA0YmnNZYkzjkfgN4ev-hXSrak_BSey_YEG6kIuDG...        tpp   
3         2U4GBNA0YmnNZYkzjkfgN4ev-hXSrak_BSey_YEG6kIuDG...        tpp   
4         2U4GBNA0YmnNZYkzjkfgN4ev-hXSrak